In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin
!pip install webdriver_manager
!pip install chromedriver_autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
from itertools import groupby

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

In [ ]:
# 사이트 생성 - 시간이 좀 걸리는군
driver = webdriver.Chrome(options = chrome_options)
driver.get("https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex=1&src=&kind=&searchCnd=1&searchStr=")
print(driver.title)

금융용어사전(목록) | 슬기로운 금융생활 길라잡이 | 금융소비자 정보포털 파인


가져와야 할 것들
- 단어
    - //*[@id="content"]/div[2]/dl[1]/dt
    - //*[@id="content"]/div[2]/dl[2]/dt
    ...
    - //*[@id="content"]/div[2]/dl[10]/dt
- 설명
    - //*[@id="content"]/div[2]/dl[1]/dd
    - //*[@id="content"]/div[2]/dl[3]/dd
    - //*[@id="content"]/div[2]/dl[10]/dd
- 단어 개수는.. 총 538개 단어가 54페이지에 걸쳐 있음 -> 대충 1~10까지 반복문 돌려놓고 없으면 패스하는 형식으로 하면 될 듯 (모두 10개씩 있고 마지막 페이지(54번째)에만 8개 있다

- 페이지
    - https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex=1&src=&kind=&searchCnd=1&searchStr=
    - https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex=2&src=&kind=&searchCnd=1&searchStr=
    - https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex=54&src=&kind=&searchCnd=1&searchStr=

    -> 1부터 54까지 바꿔서 넣어가면서 페이지 새로 생성하는 게 낫겠다

- '다음 페이지' 버튼 눌러가면서 넘어가는 게 더 효율적인..? 방법일지도..
    - //*[@id="content"]/div[3]/div/ul/li[13]/a
    - 근데 마지막 몇 페이지에서는 저 버튼이 없음... ㅇㅅㅇ

In [ ]:
# 첫 페이지로 테스트
word = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/dl[1]/dt')
print(word.text)
definition = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/dl[1]/dd')
print(definition.text)

538. 휴면예금 [Dormant Deposit]
은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸시효(은행예금 5년, 우체국예금 10년)가 완성된 이후에 찾아가지 않은 예금이다. 고객은 자신의 휴면예금을 전국은행연합회 홈페이지(http://www.sleepmoney.or.kr)의 통합조회시스템을 이용하여 확인하거나 또는 가까운 은행 방문을 통해 확인할 수 있으며, 휴면계좌가 존재하는 경우 고객은 해당 금융기관으로 직접 방문하여 정해진 절차에 따라 수령할 수 있다. 2006년 4월 27일 구축된 통합조회시스템은 2003년 1월 1일 이후의 휴면예금을 대상으로 하고 있다.


In [59]:
# 한 페이지에서 단어랑 뜻 긁어오는 함수 (+ 단어 번호랑 페이지 번호 저장)
def get_financial_words(page_num):
    # 54페이지를 제외하면 10개씩
    if page_num < 54:
        driver = webdriver.Chrome(options = chrome_options)
        driver.get(f"https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex={page_num}&src=&kind=&searchCnd=1&searchStr=")

        for i in range(1, 11):
            word_path = f'//*[@id="content"]/div[2]/dl[{i}]/dt'
            definition_path = f'//*[@id="content"]/div[2]/dl[{i}]/dd'

            word_w_num = driver.find_element(By.XPATH, word_path)
            num, word = word_w_num.text.split(".")
            matching = re.search(r"\[(.*?)\]", word) # . : 임의의 문자 한 개, * : 앞의 패턴을 여러 번 반복, ? : 최소한으로만 매칭
            eng_word = matching.group(1) if matching else ""
            word = re.sub(r"\[(.*?)\]", "", word).strip()
            words.append(word)
            nums.append(num)
            definition = driver.find_element(By.XPATH, definition_path)
            definitions.append(definition.text + " " + eng_word)

            pages.append(page_num)

    else:
        driver = webdriver.Chrome(options = chrome_options)
        driver.get(f"https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex={page_num}&src=&kind=&searchCnd=1&searchStr=")

        for i in range(1, 9):
            word_path = f'//*[@id="content"]/div[2]/dl[{i}]/dt'
            definition_path = f'//*[@id="content"]/div[2]/dl[{i}]/dd'

            word_w_num = driver.find_element(By.XPATH, word_path)
            num, word = word_w_num.text.split(". ")
            matching = re.search(r"\[(.*?)\]", word) # . : 임의의 문자 한 개, * : 앞의 패턴을 여러 번 반복, ? : 최소한으로만 매칭
            eng_word = matching.group(1) if matching else ""
            word = re.sub(r"\[(.*?)\]", "", word).strip()
            words.append(word)
            nums.append(num)
            definition = driver.find_element(By.XPATH, definition_path)
            definitions.append(definition.text + " " + eng_word)

            pages.append(page_num)

    return pages, nums, words, definitions

In [60]:
pages = []
nums = []
words = []
definitions = []

# 54페이지 -> 2분 소요
for page_num in range(1, 55):
    pages, nums, words, definitions = get_financial_words(page_num)
    print(f"{page_num} 페이지 완료 !")

financial_words = pd.DataFrame({'페이지' : pages, '번호' : nums,'용어': words, '의미': definitions})

1 페이지 완료 !
2 페이지 완료 !
3 페이지 완료 !
4 페이지 완료 !
5 페이지 완료 !
6 페이지 완료 !
7 페이지 완료 !
8 페이지 완료 !
9 페이지 완료 !
10 페이지 완료 !
11 페이지 완료 !
12 페이지 완료 !
13 페이지 완료 !
14 페이지 완료 !
15 페이지 완료 !
16 페이지 완료 !
17 페이지 완료 !
18 페이지 완료 !
19 페이지 완료 !
20 페이지 완료 !
21 페이지 완료 !
22 페이지 완료 !
23 페이지 완료 !
24 페이지 완료 !
25 페이지 완료 !
26 페이지 완료 !
27 페이지 완료 !
28 페이지 완료 !
29 페이지 완료 !
30 페이지 완료 !
31 페이지 완료 !
32 페이지 완료 !
33 페이지 완료 !
34 페이지 완료 !
35 페이지 완료 !
36 페이지 완료 !
37 페이지 완료 !
38 페이지 완료 !
39 페이지 완료 !
40 페이지 완료 !
41 페이지 완료 !
42 페이지 완료 !
43 페이지 완료 !
44 페이지 완료 !
45 페이지 완료 !
46 페이지 완료 !
47 페이지 완료 !
48 페이지 완료 !
49 페이지 완료 !
50 페이지 완료 !
51 페이지 완료 !
52 페이지 완료 !
53 페이지 완료 !
54 페이지 완료 !


In [62]:
# 저장용 데이터프레임
financial_words.to_pickle("/content/drive/MyDrive/3. Grad School/Lab Meetings/2025.08.29/Instruction Tuning/DATA/Financial_Words.pickle")

In [63]:
# 튜닝에 사용할 json
import json
financial_json = [{"용어" : word, "의미" : definition} for word, definition in zip(words, definitions)]

with open("/content/drive/MyDrive/3. Grad School/Lab Meetings/2025.08.29/Instruction Tuning/DATA/Financial_Words.json", "w", encoding = "utf-8") as f:
    json.dump(financial_json, f, ensure_ascii = False)